<a href="https://colab.research.google.com/github/noahvlone/sistem-pendukung-keputusan/blob/main/Sistem_Pendukung_Keputusan_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("NIM         : 4522210103")
print("Nama        : Farhan Ramadhan")
print("Mata Kuliah : Sistem Pendukung Keputusan A")

NIM         : 4522210103
Nama        : Farhan Ramadhan
Mata Kuliah : Sistem Pendukung Keputusan A


##Import Libraries

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import yfinance as yf

##Fetch Historical Stocks Data

In [2]:
# List of selected stocks (10 global stocks)
stocks = ['MSFT', 'TSLA', 'AMZN', 'NVDA', 'SONY', 'AAPL', 'CSCO', 'META', 'IBM', 'INTC']

# Fetch historical stock data
print("\nFetching data from Yahoo Finance...")
data = yf.download(stocks, start='2024-11-01', end='2024-12-01')


Fetching data from Yahoo Finance...


[*********************100%***********************]  10 of 10 completed


##Calculate Criteria Values

In [3]:
# Calculate daily returns
returns = data['Adj Close'].pct_change().dropna()

# Calculate criteria values
volatility = returns.std() * np.sqrt(252)  # Annualized volatility (Risk)
average_return = returns.mean() * 252      # Annualized return
average_volume = data['Volume'].mean()     # Liquidity

# Display computed criteria
print("\n--- Computed Criteria Values ---")
criteria_df = pd.DataFrame({
    'Risk (Volatility)': volatility,
    'Return': average_return,
    'Liquidity (Volume)': average_volume
})
print(criteria_df)


--- Computed Criteria Values ---
        Risk (Volatility)    Return  Liquidity (Volume)
Ticker                                                 
AAPL             0.142515  0.856979          44582030.0
AMZN             0.319060  0.700217          45296895.0
CSCO             0.172000  0.873630          20497360.0
IBM              0.219702  1.297332           3863975.0
INTC             0.502130  0.595856          71765820.0
META             0.245753  0.195137          11890560.0
MSFT             0.189151  0.460352          22116060.0
NVDA             0.393233  0.349433         224903765.0
SONY             0.408667  1.713046           3115575.0
TSLA             0.829355  4.684187         104595695.0


##Define AHP Pairwise Comparison Matrix

In [4]:
# Define AHP pairwise comparison matrix for criteria
criteria_matrix = np.array([
    [1, 3, 5],  # Risk is moderately to strongly more important than Return and Liquidity
    [1/3, 1, 3], # Return is less important than Risk but more than Liquidity
    [1/5, 1/3, 1]  # Liquidity is least important
])

# Function to calculate weights from AHP matrix
def calculate_weights(matrix):
    column_sums = matrix.sum(axis=0)
    normalized_matrix = matrix / column_sums
    weights = normalized_matrix.mean(axis=1)
    return weights

# Calculate criteria weights
criteria_weights = calculate_weights(criteria_matrix)
print("\n--- Criteria Weights ---")
print(criteria_weights)


--- Criteria Weights ---
[0.63334572 0.26049796 0.10615632]


##Normalize and Calculate Overall Scores

In [5]:
# Normalize the alternatives for each criterion
def normalize_criteria(values):
    return values / values.sum()

# Normalize the criteria
normalized_risk = normalize_criteria(1 / criteria_df['Risk (Volatility)'])  # Lower risk is better
normalized_return = normalize_criteria(criteria_df['Return'])              # Higher return is better
normalized_liquidity = normalize_criteria(criteria_df['Liquidity (Volume)']) # Higher liquidity is better

# Combine normalized scores with criteria weights
overall_scores = (
    criteria_weights[0] * normalized_risk +
    criteria_weights[1] * normalized_return +
    criteria_weights[2] * normalized_liquidity
)

# Add scores to the dataframe
criteria_df['Overall Score'] = overall_scores
criteria_df = criteria_df.sort_values(by='Overall Score', ascending=False)

##Display Results

In [6]:
# Display title
print("\n--- Decision Support System for Stock Investment ---")
print("\n---      Criteria: Risk, Return, Liquidity       ---")

# Display results
print("\nInvestment Recommendation")
print(criteria_df)

# Display Top 3 stocks
print("\nTop 3 Stocks to Invest In:")
print(criteria_df.head(3))


--- Decision Support System for Stock Investment ---

---      Criteria: Risk, Return, Liquidity       ---

Investment Recommendation
        Risk (Volatility)    Return  Liquidity (Volume)  Overall Score
Ticker                                                                
AAPL             0.142515  0.856979          44582030.0       0.144367
TSLA             0.829355  4.684187         104595695.0       0.144217
CSCO             0.172000  0.873630          20497360.0       0.120094
IBM              0.219702  1.297332           3863975.0       0.105305
MSFT             0.189151  0.460352          22116060.0       0.102452
NVDA             0.393233  0.349433         224903765.0       0.093283
SONY             0.408667  1.713046           3115575.0       0.079374
AMZN             0.319060  0.700217          45296895.0       0.076412
META             0.245753  0.195137          11890560.0       0.074333
INTC             0.502130  0.595856          71765820.0       0.060163

Top 3 Stocks

##Save Data

In [7]:
# Save historical stock data to a CSV file
historical_data_filename = 'historical_stock_data.csv'
data.to_csv(historical_data_filename)
print(f"\nHistorical stock data saved to: {historical_data_filename}")

# Save criteria dataframe to a CSV file
criteria_data_filename = 'criteria_data.csv'
criteria_df.to_csv(criteria_data_filename)
print(f"Criteria and overall scores saved to: {criteria_data_filename}")


Historical stock data saved to: historical_stock_data.csv
Criteria and overall scores saved to: criteria_data.csv
